# 程式說明
作業環境: `jupyter notebook`

使用語言:`Python3`

請安裝`pip install twstock`   目前使用版本1.3.1

### 拒絕存取應對方法:

**注意:五秒鐘內連續存取會被ban掉**
1.  更換IP:手機連線轉飛航模式後再切回來
2.  增加連線時間:透過 `import time` 的`time sleep(5)` 
3.  VPN、無痕瀏覽的方法

----
### 一、讀取.txt函式
* 1          將讀取寫成函式方便使用。
* 3~12   為`try...except...`區塊，用來處理在開讀檔時可能遇到的錯誤。
* 4~9     為開啟檔案的`with`區塊，在區塊結束時自動關閉檔案
* 4          由於文字檔裡面只剩下逗號，因此開檔時並未指定編碼(使用預設的`cp950`或是`utf-8`開都可以)

In [1]:
def get_setting(): 
    res = [ ] #準備一個空的list來存放讀取及解析的結果
    try :
        with open('stock.txt',encoding='utf-8') as f : #用with 以讀取模式開啟檔案
            slist = f.readlines()             #以行為單位讀取所有資料
            print('讀入:',slist )                  #輸出讀到的資料以供確認
            for lst in slist :                       #用for迴圈走訪每一個單位
                s = lst.split(',')                   #將股票以逗號切割為串列
                res.append([s[0].strip(),   float(s[1]),float(s[2])])
               # ^將切割結果加到res^去除左右空白^^最後再將股價轉換為float
        
    except:
        print('stock.txt 讀取錯誤')
    return  res  
    #傳回解析結果，但如果開檔或讀檔錯誤則會回傳[ ]

stock = get_setting()#呼叫上面函式
print('傳回:',stock)     #輸出傳回的結果


讀入: ['2330,220,223\n', '2317,66,69\n', '1301,97,100']
傳回: [['2330', 220.0, 223.0], ['2317', 66.0, 69.0], ['1301', 97.0, 100.0]]


---
### 二、查詢歷史資料

* date:
     - 日期(datetime.datetime格式)
* capacity:
     - 總成交股數(單位:股)
* turnover:
     - 總成交金額(單位:新台幣/元)
* open:
     - 開盤價
* high:
     - 盤中最高價
* low:
     - 盤中最低價
* price:
     - 收盤價
*  close: 
     - 收盤價(同上)
* change:
     - 漲跌價差
* transaction:
     - 成交筆數

In [2]:
import twstock
import time

stock = twstock.Stock('2330')#以台積電的股票代號 建立`Stock`物件
print(stock.price)                    #輸出最近31天的收盤價

[249.0, 242.5, 244.0, 243.0, 242.5, 242.0, 247.0, 250.0, 250.5, 254.5, 256.0, 252.0, 254.0, 259.0, 264.0, 264.0, 265.0, 265.0, 261.0, 261.0, 260.0, 259.5, 256.5, 251.5, 246.5, 248.5, 248.0, 253.5, 251.0, 246.5, 249.5]


輸出最近一日的資料

In [8]:
print('日期:',stock.date[-1])
print('開盤價:',stock.open[-1])
print('最高價:',stock.high[-1])
print('最低價:',stock.low[-1])

日期: 2019-08-14 00:00:00
開盤價: 252.5
最高價: 254.0
最低價: 249.5


使用 `fetch()`可以取得指定區間的歷史資料，再用上一個方法讀取內容。

In [22]:
stock = twstock.Stock('2330')
#stock.fetch(2019,5)                #2019年五月份的資訊
#stock.fetch_31()                      #最近31天的資料
#stock.fetch_from(2018,9)        # 取得2019,9月至今的資料

---
### 三、 股票分析與四大賣點

#### 四大賣點原則為:(小括號中為賣點)
1. 量大收紅(黑)
2. 量縮價不跌(價跌)
3. 三日均價由下往上(由上往下)
4. 三日均價大於(小於)六日均價

#### 移動平均(wiki)
[https://zh.wikipedia.org/wiki/%E7%A7%BB%E5%8B%95%E5%B9%B3%E5%9D%87 ]

In [4]:
import twstock
stock = twstock.Stock('2330')
print(stock.moving_average(stock.price,5))#計算每天的五日移動平均收盤價

[244.2, 242.8, 243.7, 244.9, 246.4, 248.8, 251.6, 252.6, 253.4, 255.1, 257.0, 258.6, 261.2, 263.4, 263.8, 263.2, 262.4, 261.3, 259.6, 257.7, 254.8, 252.5, 250.2, 249.6, 249.5, 249.5, 249.7]


#### 呼叫`BestFourPoint()`

如果是買點，會回傳(`True`，符合的原則)，若賣點回傳(`False`，符合的原則)，如果都不符合則回傳`None`

In [6]:
bfp = twstock.BestFourPoint(stock)#以stock建立四大賣點
print(bfp.best_four_point())          #判斷是否為四大賣點

(False, '量大收黑, 三日均價小於六日均價')


---
### 四、查詢即時交易資訊

In [12]:
import twstock
rt = twstock.realtime.get('2330') #取得台積電股票
if  ( rt ['success'] ):                         #如果讀取成功(rt 為一個dict)
    print(rt)

{'timestamp': 1565764200.0, 'info': {'code': '2330', 'channel': '2330.tw', 'name': '台積電', 'fullname': '台灣積體電路製造股份有限公司', 'time': '2019-08-14 14:30:00'}, 'realtime': {'latest_trade_price': '249.50', 'trade_volume': '4831', 'accumulate_trade_volume': '35180', 'best_bid_price': ['249.50', '249.00', '248.50', '248.00', '247.50'], 'best_bid_volume': ['3044', '1077', '740', '1066', '589'], 'best_ask_price': ['250.00', '250.50', '251.00', '251.50', '252.00'], 'best_ask_volume': ['403', '566', '435', '1344', '1694'], 'open': '252.50', 'high': '254.00', 'low': '249.50'}, 'success': True}


---
# 完整程式碼

#### 程式說明:
* 5~7 判斷如果讀取成功，就回傳內容為(股票名稱，即時價格)的list，讀取失敗就回傳(False,False)

* 13 函式會回傳(Ture 或 False ，符合的原則)，若未符合買點則回傳None

* 14~15 判斷若為bp為真 (不為None)，就再判斷bp[0]為真時傳回('買進',bp[1]),其中的bp[1]為符合原則。

* 19 由於`get_price()`會傳回2個元素的tuple,因此等號左邊可以用2個變數來承接(多重指定,參見第二章補充學習)

* 20 `get_best()`同樣會傳回2個元素的tuple,因此用act及note變數來承接。

In [25]:
import twstock

def get_price(stockid):                             # 取得股票名稱和及時股價
    rt = twstock.realtime.get(stockid)   # 取得台積電的及時交易資訊
    if rt['success']:                                         # 如果讀取成功
        return (rt['info']['name'],                 #傳回 (股票名稱, 及時價格)
                float(rt['realtime']['latest_trade_price']))
    else:
        return (False, False)

def get_best(stockid):                             # 檢查是否符合四大買賣點
    stock = twstock.Stock(stockid)
    bp = twstock.BestFourPoint(stock).best_four_point()
    if(bp):
        return ('買進' if bp[0] else '賣出', bp[1])  #傳回買進或賣出的建議
    else:
        return (False, False)                 #都不符合

name, price = get_price('2330')  #用 name 及 price 來承接傳回的 tuple
act, why = get_best('2330')          #用 act 及 why 來承接傳回的四大買賣點 tuple
print(name, price, act, why, sep=' | ')


台積電 | 249.5 | 賣出 | 量大收黑, 三日均價小於六日均價


# 匯入IFTTT
[https://ifttt.com ]
--- 
用 `requests`模組發送Line通知

* 4~8 將IFTTT的網址與參數串接起來，再串接前後多加了小括號以方便寫成多行的可讀性
* 10~11 檢查IFTTT的回應文字，判斷是否傳送成功

In [34]:
import requests      # 匯入 requests 套件

def send_ifttt(v1, v2, v3):   # 定義函式來向 IFTTT 發送 HTTP 要求
    url = ('https://maker.ifttt.com/trigger/toline/with/' +
          'key/cXbOXFY9BsXqqGBaAw-5NO' +
          '?value1='+str(v1) +
          '&value2='+str(v2) +
          '&value3='+str(v3))
    r = requests.get(url)      # 送出 HTTP GET 並取得網站的回應資料
    if r.text[:5] == 'Congr':  # 回應的文字若以 Congr 開頭就表示成功了
        print('已傳送 (' +str(v1)+', '+str(v2)+', '+str(v3)+ ') 到 Line')
    return r.text

ret = send_ifttt('包子', 999999, '高雄發大財')  #傳送 HTTP 請求到 IFTTT
print('IFTTT 的回應訊息：', ret)     # 輸出 IFTTT 回應的文字


已傳送 (包子, 999999, 高雄發大財) 到 Line
IFTTT 的回應訊息： Congratulations! You've fired the toline event


In [ ]:
#------------------------------------------------------------------------------------
# 函式總整理，可以直接copy轉成自訂模組stock_module.py
# import  stock_module as m 
#也可用 slist = m.get_seting()  #呼叫指定的方法
#------------------------------------------------------------------------------------

def get_price(stockid):                             # 取得股票名稱和及時股價
    rt = twstock.realtime.get(stockid)   # 取得台積電的及時交易資訊
    if rt['success']:                                         # 如果讀取成功
        return (rt['info']['name'],                 #傳回 (股票名稱, 及時價格)
                float(rt['realtime']['latest_trade_price']))
    else:
        return (False, False)

def get_best(stockid):                             # 檢查是否符合四大買賣點
    stock = twstock.Stock(stockid)
    bp = twstock.BestFourPoint(stock).best_four_point()
    if(bp):
        return ('買進' if bp[0] else '賣出', bp[1])  #傳回買進或賣出的建議
    else:
        return (False, False)                 #都不符合




def send_ifttt(v1, v2, v3):   # 定義函式來向 IFTTT 發送 HTTP 要求
    url = ('https://maker.ifttt.com/trigger/toline/with/' +
          'key/cXbOXFY9BsXqqGBaAw-5NO' +
          '?value1='+str(v1) +
          '&value2='+str(v2) +
          '&value3='+str(v3))
    r = requests.get(url)      # 送出 HTTP GET 並取得網站的回應資料
    if r.text[:5] == 'Congr':  # 回應的文字若以 Congr 開頭就表示成功了
        print('已傳送 (' +str(v1)+', '+str(v2)+', '+str(v3)+ ') 到 Line')
    return r.text

def get_setting(): 
    res = [ ] #準備一個空的list來存放讀取及解析的結果
    try :
        with open('stock.txt',encoding='utf-8') as f : #用with 以讀取模式開啟檔案
            slist = f.readlines()             #以行為單位讀取所有資料
            print('讀入:',slist )                  #輸出讀到的資料以供確認
            for lst in slist :                       #用for迴圈走訪每一個單位
                s = lst.split(',')                   #將股票以逗號切割為串列
                res.append([s[0].strip(),   float(s[1]),float(s[2])])
               # ^將切割結果加到res^去除左右空白^^最後再將股價轉換為float
        
    except:
        print('stock.txt 讀取錯誤')
    return  res  
    #傳回解析結果，但如果開檔或讀檔錯誤則會回傳[ ]

    
#--------------------------------------------------------------------------------------------------------------
import time          #避免被BAN掉 ，使用sleep()來調整時間
import requests
import twstock

slist = get_setting()#呼叫匯入模組中的函式取得股票設定資料
cnt = len(slist)          #計算有幾支股票

log1 = []                       #紀錄曾經傳送過的股票高或低於期望價的訊息，以避免重複傳送
log2 = []                        #紀錄曾經傳送過符合四大買賣點的訊息，以避免重複傳送

for i in range(cnt):           #為每支股票加入對應的訊息
    log1.append('')
    log2.append('')

check_cnt = 20               # 要檢查幾次(20*3分鐘 = 60分鐘)
while True:
    for i in range(cnt):   # 走訪每一支股票
        id , low , high = slist [ i ] #讀出股票的代號、期望買進價格、期望賣出價格
        name , price = get_price(id) #讀取股票名稱和即時價格
        print('檢查 :' ,name ,'股價 :' ,price,'區間 :',low,'~',high)
        if price <= low :  # 如果即時股票到達期望買點
            if log1[ i ]  != '買進':#檢查前一次傳送訊息，避免重複傳送
                send_ifttt(name,price,'買進(股價低於'+str(low)+')')
                log1[i] = '買進'  #紀錄傳送訊息，以避免重複傳送
            
        elif price >= high :#如果即時股價到達期望賣點
            if log1[i]  != '賣出':#檢查前一次傳送訊息
                send_ifttt(name,price,'賣出(股價高於'+str(low)+')')
                log1[i] = '賣出' #紀錄傳送訊息避免重複傳送
        act , why = get_best(id)#檢查四大賣點
        if why :                             #如果符合四大賣點
            if log2[i] != why:
                send_ifttt(name ,price ,act + '('+why+')' )
                log2[i ] = why   #同上記錄與檢查
    
    print('---------------------')
    check_cnt -= 1                                #將計數器減1
    if check_cnt == 0:break              # 檢查計數器為0時即離開迴圈，結束程式
    time.sleep(180)                              #每3秒檢查一次

讀入: ['2330,220,223\n', '2317,66,69\n', '1301,97,100']
檢查 : 台積電 股價 : 247.0 區間 : 220.0 ~ 223.0
已傳送 (台積電, 247.0, 賣出(股價高於220.0)) 到 Line
已傳送 (台積電, 247.0, 賣出(量大收黑, 三日均價小於六日均價)) 到 Line
檢查 : 鴻海 股價 : 71.6 區間 : 66.0 ~ 69.0
已傳送 (鴻海, 71.6, 賣出(股價高於66.0)) 到 Line
已傳送 (鴻海, 71.6, 賣出(量大收黑, 三日均價小於六日均價)) 到 Line
檢查 : 台塑 股價 : 91.3 區間 : 97.0 ~ 100.0
已傳送 (台塑, 91.3, 買進(股價低於97.0)) 到 Line
已傳送 (台塑, 91.3, 賣出(量縮價跌, 三日均價小於六日均價)) 到 Line
---------------------
檢查 : 台積電 股價 : 247.0 區間 : 220.0 ~ 223.0
檢查 : 鴻海 股價 : 71.7 區間 : 66.0 ~ 69.0
檢查 : 台塑 股價 : 91.2 區間 : 97.0 ~ 100.0
---------------------
